In [5]:
import random as rd
from math import *


def max_pui_deux(n):
    if n == 0:
        return 0
    i = 1
    while n % i != 0:
        i *= 2
    return i


'''
    Compute x^n
    Parameters :
    - A number x
    - A power n
'''


def quickExponent(x, n):
    # Base x cases
    if x == 0:
        return 0
    if x == 1:
        return 1
    # Base n cases
    if n == 0:
        return 1
    if n == 1:
        return x
    # Heredity
    if (n % 2 == 0):
        return (quickExponent(x, n//2) ** 2)
    else:
        return (quickExponent(x, n//2) ** 2) * x


'''
    Compute x^n %m
    Parameters :
        - A number x
        - A power n
        - A modulus m
'''


def quickModularExponent(x, n, m):
    # Base x cases
    if x == 0:
        return 0
    if x == 1:
        return 1
    # Base n cases
    if n == 0:
        return 1
    if n == 1:
        return x % m
    # modulo 1 case
    if m == 1:
        return 1
    # Heredity
    if (n % 2 == 0):
        return (quickModularExponent(x, n//2, m) ** 2) % m
    else:
        return ((quickModularExponent(x, n//2, m) ** 2) * x) % m


def miller_rabin(n, k, m):
    if n == 1:
        return False
    if n == 2:
        return True
    # n = 1 + m*2^k, where m is odd,
    # i.e. 2^k is the highest power of 2 dividing n-1, m being the other factor
    a = rd.randint(2, n-1)
    if quickModularExponent(a, m, n) == 1 or quickModularExponent(a, m, n) == n-1:
        return True
    for i in range(1, k):
        if quickModularExponent(a, 2*i*m, n) == n-1:
            return True
    return False


def isProbablyPrime(n, likelihood=99):
    k = max_pui_deux(n-1)
    m = int((n-1)/(quickExponent(2, k)))
    # Base error rate for a single miller-rabin test is about 1/4.
    # So false-positive probability after j tests is 1 - 4^(-j)
    # To reach the given likelihood l, we need 1 - 4^(-j) >= l/100
    # 4^(-j) <= (100-l)/100
    # -j <= log4(100 - l) - log4(100)
    # j >= log4(100) - log4(100 - l)
    # log4(100) is about 3.32, let's round it to 4
    for i in range(floor(4 - log(100 - likelihood, 4))):
        if not miller_rabin(n, k, m):
            return False
    return True


''' 
    TESTS
'''


def main():
    print(quickModularExponent(78, 4567, 154))

    print(miller_rabin(33, 4, 2))  # 33 is 2*2^4 + 1
    print(miller_rabin(31, 1, 15))  # 31 is 15*2^1 + 1
    print(miller_rabin(17, 3, 2))  # 17 is 2*2^3 + 1
    print(miller_rabin(3, 1, 1))  # 3 is 1*2^1 + 1 ???

    print(isProbablyPrime(3))
    print(isProbablyPrime(17))
    print(isProbablyPrime(31))


if __name__ == "__main__":
    main()


78
True
True
True
True
True
True
True


In [4]:
from re import M
from MR import isProbablyPrime, quickModularExponent
from math import gcd
import random

'''
    Global variables for key size
'''
min = 2**32
max = 2**64-1

'''
    Generates a prime number. Uses Miller-Rabin as primality test.
    Parameters :
        - a minimal power of 2 min
        - a maximal power of 2 max
'''


def generate_prime():
    r = random.randint(min, max)
    while(not isProbablyPrime(r)):
        r = random.randint(min, max)
    return r


'''
    Finds a number coprime with phi
    Parameters :
        - The value phi = (p-1) * (q-1)

    Returns :
        A number e such as 1 < e < phi(n) and gcd(e,phi(n)) = 1

'''


def find_coprime(phi):
    for i in range(2, phi):
        if (gcd(i, phi) == 1):
            return i
    return -1


'''
    Find the phi modular inverse of e using Euclide's algorithm
    Parameters :
        - The number e
        - The modulo phi
    Returns
        A number d such as ed = 1 mod phi
'''


def find_inverse(e, phi):
    m = phi
    x = 1
    y = 0

    if(phi == 1):
        return 0
    while (e > 1):
        q = e//phi
        t = phi

        phi, e, t = e % phi, t, y
        y, x = x-q*t, t

    if (x < 0):
        x += m
    return x


'''
    Generates public and private keys for RSA
    Parameters :
        None

    Returns :
        A list [(n,e),(d,n)] containing the public and the private keys
'''


def generate_RSA():
    p, q = generate_prime(), generate_prime()
    if (q == p):
        q = generate_prime()

    n = p*q
    phi = (p-1) * (q-1)
    e = find_coprime(phi)

    d = find_inverse(e, phi)
    return [[n, e], [d, n]]


'''
    Encrypts a message using RSA
    Parameters :
        - msg : the message to encrypt
        - key : the public key (n,e)

    Returns :
        The encrypted message
'''


def encrypt_RSA(msg, key):
    binary = bin(ord(msg[0]))
    for i in range(1, len(msg)):
        binary += bin(ord(msg[i]))[2::]

    binary = bin(quickModularExponent(int(binary, base=2), key[1], key[0]))

    cyphered = ""
    for i in range(2, len(binary), 8):
        cyphered += chr(int(binary[i:i+7],base=2))
    return cyphered


'''
    Decrypt a message using RSA
    Parameters :
        - msg : the message to decrypt
        - key : the private key (d,n)

    Returns :
        The raw message
'''


def decrypt_RSA(msg, key):
    binary = bin(ord(msg[0]))
    for i in range(1, len(msg)):
        binary += bin(ord(msg[i]))[2::]

    binary = bin(quickModularExponent(int(binary, base=2), key[0], key[1]))

    raw = ""
    for i in range(2, len(binary), 8):
       raw += chr(int(binary[i:i+7],base=2))
    return raw


'''
    TESTS
'''


def main():
    print("Generated keys : ")
    k = generate_RSA()
    print(k)

    print("Encrypt 'HELLO' :")
    m = encrypt_RSA("HELLO", k[0])
    print(m)

    print("Decrypt 'HELLO' : ")
    print(decrypt_RSA(m,k[1]))


if __name__ == "__main__":
    main()


Generated keys : 
[[148702399235689551033965794206311912, 7], [63729599672438373960595061360023963, 148702399235689551033965794206311912]]
Encrypt 'HELLO' :
N7D{t	C?Ba3
Decrypt 'HELLO' : 
BqEy8|[t w\%


In [13]:
hello = "Hello" 
res = bin(ord(hello[0]))
for i in range(1,len(hello)):
    res += bin(ord(hello[i]))[2::]
print(int(res,base = 2))


19540948591
